In [31]:
import os
import pandas as pd
from eodhd import APIClient
from datetime import datetime, timedelta
import requests
import datetime
import pytz


In [32]:
api_key = os.getenv('EODHD_API_KEY')
api_key

'67b5ff99b63306.32749083'

In [33]:
api = APIClient(api_key)
api

In [34]:
# -------------------------------
# Define the trading day time range
# -------------------------------

# US markets are listed in Eastern Time.
et = pytz.timezone("US/Eastern")

# To include pre-market (before the official 9:30 AM open), we start early (say, 7:00 AM ET).
# And we end at 4:00 PM ET, covering the entire regular session.
start_et = et.localize(datetime.datetime(2025, 5, 15, 7, 0, 0))
end_et   = et.localize(datetime.datetime(2025, 5, 15, 16, 0, 0))

# Convert both start and end times to UTC (the API requires Unix timestamps in UTC)
start_utc = start_et.astimezone(pytz.utc)
end_utc   = end_et.astimezone(pytz.utc)
from_timestamp = int(start_utc.timestamp())
to_timestamp   = int(end_utc.timestamp())

# Print the calculated times for verification (ISO format)
print("Requesting data from (UTC):", start_utc.isoformat())
print("Until (UTC):", end_utc.isoformat())


Requesting data from (UTC): 2025-05-15T11:00:00+00:00
Until (UTC): 2025-05-15T20:00:00+00:00


In [35]:
# -------------------------------
# Construct the API request URL
# -------------------------------
ticker = "AAPL.US"          # For US tickers, only NYSE and NASDAQ are supported for 1 or 5-minutes data.
url = (
    f"https://eodhd.com/api/intraday/{ticker}"
    f"?interval=1m&api_token={api_key}&fmt=json"
    f"&from={from_timestamp}&to={to_timestamp}"
)


In [36]:
# -------------------------------
# Make the API request and decode the JSON
# -------------------------------
response = requests.get(url)
# Check for errors (optional but recommended)
if response.status_code != 200:
    print("Error fetching data:", response.status_code)
    print(response.text)
    exit()

data = response.json()

In [37]:
# -------------------------------
# Convert the response into a DataFrame and make the dates readable
# -------------------------------
df = pd.DataFrame(data)
df

# # Assuming each data point has a Unix timestamp in a field (e.g., "timestamp"),
# # convert it to a datetime in UTC then convert to Eastern Time.
# df['datetime'] = pd.to_datetime(df['timestamp'], unit='s', utc=True).dt.tz_convert("US/Eastern")
# # Format the datetime as "YYYY-MM-DD HH:MM"
# df['date_readable'] = df['datetime'].dt.strftime("%Y-%m-%d %H:%M")

# # Display selected columns: human-readable date and OHLC values.
# print(df[['date_readable', 'open', 'high', 'low', 'close']])

,timestamp,gmtoffset,datetime,open,high,low,close,volume
0,1747306800,0,2025-05-15 11:00:00,209.56000000,209.70000000,209.50000000,209.70000000,5625
1,1747306860,0,2025-05-15 11:01:00,209.70000000,209.72000000,209.68000000,209.71000000,2630
2,1747306920,0,2025-05-15 11:02:00,209.71000000,209.78000000,209.70000000,209.74000000,3919
3,1747306980,0,2025-05-15 11:03:00,209.74000000,209.79000000,209.70000000,209.78000000,1088
4,1747307040,0,2025-05-15 11:04:00,209.76000000,209.84000000,209.70000000,209.70000000,2406
...,...,...,...,...,...,...,...,...
536,1747338960,0,2025-05-15 19:56:00,210.85000000,211.07000000,210.85000000,211.05000000,219198
537,1747339020,0,2025-05-15 19:57:00,211.04000000,211.29000000,210.95000000,211.23500000,557600
538,1747339080,0,2025-05-15 19:58:00,211.23500000,211.24000000,211.03000000,211.10500000,410641
539,1747339140,0,2025-05-15 19:59:00,211.10500000,211.47000000,210.95000000,211.45000000,1776066
